*JAI SHREE RADHE KRISHNA*

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F


In [21]:

# hyperparameters
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------


In [22]:

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [23]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [24]:
torch.manual_seed(1337)
xb , yb =get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('Targets:')
print(yb.shape)
print(yb)
print('-'*100)
for b in range(batch_size):
    for t in range(batch_size+1):
      context = xb[b,:t+1]
      target = yb[b,t]
      print(f"When input is {context.tolist()}, the target is {target}")

inputs
torch.Size([4, 8])
tensor([[38,  1, 57, 47, 53,  1, 55, 37],
        [37, 50, 37,  4,  0, 10, 46, 36],
        [45, 57,  1, 38, 33, 52, 40, 37],
        [ 1, 40, 33, 52, 40,  1, 34, 50]])
Targets:
torch.Size([4, 8])
tensor([[ 1, 57, 47, 53,  1, 55, 37, 50],
        [50, 37,  4,  0, 10, 46, 36,  1],
        [57,  1, 38, 33, 52, 40, 37, 50],
        [40, 33, 52, 40,  1, 34, 50, 37]])
----------------------------------------------------------------------------------------------------
When input is [38], the target is 1
When input is [38, 1], the target is 57
When input is [38, 1, 57], the target is 47
When input is [38, 1, 57, 47], the target is 53
When input is [38, 1, 57, 47, 53], the target is 1
When input is [37], the target is 50
When input is [37, 50], the target is 37
When input is [37, 50, 37], the target is 4
When input is [37, 50, 37, 4], the target is 0
When input is [37, 50, 37, 4, 0], the target is 10
When input is [45], the target is 57
When input is [45, 57], the tar

In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)

            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx
    

m=BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([256, 59])
tensor(4.6577, grad_fn=<NllLossBackward0>)

HmtrDri-HioexqvyfUphVtF
KQmerBA? h
jNU?gqihiYbot,KERFmopuC': MS.Nsv rq!Bik
WviMx;.eRV
G-,qsPqzMYyu-T


In [48]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [51]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    # print(xb,yb)
    logits, loss = m(xb,yb)
    # print(logits,loss)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())    

2.30324649810791


In [54]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long),max_new_tokens=500)[0].tolist()))



y chexte, ls hefe ay s s'sposa od,

I ouse avorerelaur;' k, y st hexthemy hed and yol my tomonar CI tharge seen f monon

I r;

N tiftou'th Gole uche inlsau?
Anty he.
Am
Tofughalas st, I islfraitor omaghin ouanoarer t nk.
Hirfily d inspouf ave ffantom lfous Goncobanguanomat s. inofustoraveessond

Thecece dy re the wfow.
Won Wiminde, tanllllllle GRDIO:
IONCantowad bemimepe! Clano ma- w tace;
LULE: he of y ss.
I hey bl t f, imupr mar-f I len!

IAhisurk; ce fan be haseThim h VANThesthemyed y m seld
